In [2]:
import cv2
import numpy as np
import os
import pandas as pd
from pathlib import Path

data_path = Path.cwd() / "data"


low_res_path = data_path /"Test Images/Low Res"
original_path = data_path/"Test Images/Original"
model_output_path = data_path/"Test Images/Model Output"

# Function to calculate Mean SSE (MSE)
def calculate_mse(imageA, imageB):
    return np.mean((imageA.astype("float") - imageB.astype("float")) ** 2)

results = []

for filename in os.listdir(low_res_path):
    if filename.endswith(".png"):
        low_res_img = cv2.imread(os.path.join(low_res_path, filename))
        model_img = cv2.imread(os.path.join(model_output_path, filename))

        # Remove 'x2' suffix to find the corresponding high-res image in the "Original" folder
        original_filename = filename.replace("x2", "")
        high_res_img = cv2.imread(os.path.join(original_path, original_filename))

        # Check if images are loaded successfully
        if low_res_img is None or high_res_img is None or model_img is None:
            print(f"Error loading one or more images for {filename}. Skipping this file.")
            continue

        # Get dimensions for upscaling
        height, width = high_res_img.shape[:2]
        
        # Upscale using different interpolation methods
        bilinear_upscaled = cv2.resize(low_res_img, (width, height), interpolation=cv2.INTER_LINEAR)
        bicubic_upscaled = cv2.resize(low_res_img, (width, height), interpolation=cv2.INTER_CUBIC)
        nearest_upscaled = cv2.resize(low_res_img, (width, height), interpolation=cv2.INTER_NEAREST)

        # Calculate Mean SSE for each method
        mse_bilinear = calculate_mse(bilinear_upscaled, high_res_img)
        mse_bicubic = calculate_mse(bicubic_upscaled, high_res_img)
        mse_nearest = calculate_mse(nearest_upscaled, high_res_img)
        mse_model = calculate_mse(model_img, high_res_img)

        # Store results
        results.append({
            "Image": filename,
            "MSE_Bilinear": mse_bilinear,
            "MSE_Bicubic": mse_bicubic,
            "MSE_Nearest": mse_nearest,
            "MSE_Model": mse_model
        })

# Create a DataFrame for better readability
results_df = pd.DataFrame(results)
print(results_df)


        Image  MSE_Bilinear  MSE_Bicubic  MSE_Nearest   MSE_Model
0  0804x2.png     85.610879    63.311168   100.479395   41.902860
1  0809x2.png     20.134811    11.800485    28.491521    7.892705
2  0820x2.png    184.990433   128.118131   219.422703   56.260582
3  0846x2.png    185.966193   142.388565   199.560043   71.988089
4  0850x2.png     71.835322    56.801367    78.044734   32.280629
5  0855x2.png     52.386281    40.660039    58.252612   31.054350
6  0866x2.png    130.443994   102.415332   137.248531   85.694887
7  0869x2.png    198.582595   158.899287   195.781864  103.219478
8  0874x2.png     48.817761    37.551371    52.999818   24.792822
9  0888x2.png     50.707544    36.718218    68.505541   24.075763
